In [1]:
import pandas as pd
import requests

In [124]:
%%time
d = []
d_future = []
element_result = {}
for element in requests.get(f'https://fantasy.premierleague.com/api/bootstrap-static/').json()["elements"]:
    element_result_ = requests.get(f'https://fantasy.premierleague.com/api/element-summary/{element["id"]}/').json()
    element_result[element["id"]] = element_result_
    for history in element_result_["history"]:
        d_= {
            "element_id": element["id"],
            "fixture_id": history['fixture'],
            "total_points": history['total_points']
        }
        d.append(d_)
    for fixture in element_result_["fixtures"]:
        d_= {
            "element_id": element["id"],
            "fixture_id": fixture['id'],
            "total_points": None
        }
        d_future.append(d_)

CPU times: user 30.9 s, sys: 742 ms, total: 31.6 s
Wall time: 1min 2s


In [126]:
bootstrap_static_result = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').json()
fixtures_result =  requests.get('https://fantasy.premierleague.com/api/fixtures/').json()
element_result = element_result 

In [130]:
# %%time
df = pd.DataFrame(d + d_future)
df["event_id"] = df["fixture_id"].apply(lambda x: next(f for f in fixtures_result if f["id"] == x)["event"])
# df["team_id"] = df.apply(lambda x: next(e for e in bootstrap_static_result["elements"] if e["id"] == x["element_id"])["team"], axis=1)
# df["opponent_team_id"] = df.apply(lambda x: next(h for h in element_result[x["element_id"]]["history"] if h["fixture"] == x["fixture_id"])["opponent_team"], axis=1)
# df["opponent_team_strength"] = df.apply(lambda x: next(t for t in bootstrap_static_result["teams"] if t["id"] == x["opponent_team_id"])["strength"], axis=1)
df["was_home"] = df.apply(lambda x: next(h for h in element_result[x["element_id"]]["history"] if h["fixture"] == x["fixture_id"])["was_home"], axis=1)
df["opponent_team_strength"] = df.apply(lambda x: next(t for t in bootstrap_static_result["teams"] if t["id"] == x["opponent_team_id"])["strength"], axis=1)
df["opponent_team_strength_overall_home"] = df.apply(lambda x: next(t for t in bootstrap_static_result["teams"] if t["id"] == x["opponent_team_id"])["strength_overall_home"], axis=1)

StopIteration: 

In [119]:
df_future = pd.DataFrame(d_future)
df = df.merge(df_future, how='outer')

In [89]:
df["predicted_total_points"] = "??"

In [120]:
df[df["element_id"]==216]

,element_id,fixture_id,total_points,event_id,opponent_team_id,was_home,opponent_team_strength,opponent_team_strength_overall_home,predicted_total_points
8171,216,9,None,1.0,11.0,True,4.0,1240.0,0.984064
8172,216,19,None,2.0,19.0,False,3.0,1100.0,1.047739
8173,216,26,None,3.0,12.0,True,2.0,1055.0,1.397881
8174,216,35,None,4.0,16.0,True,3.0,1065.0,1.190973
8175,216,42,None,5.0,3.0,False,3.0,1100.0,1.047739
8176,216,55,None,6.0,2.0,True,4.0,1160.0,0.984064
8177,216,64,None,7.0,10.0,False,3.0,1055.0,1.047739
8178,216,73,None,8.0,6.0,False,2.0,1045.0,1.254648
8179,216,84,None,9.0,1.0,True,5.0,1350.0,0.777155
8180,216,95,None,10.0,4.0,True,3.0,1100.0,1.190973


In [107]:
df.shape

(21427, 9)

In [101]:
import numpy as np
from sklearn.linear_model import LinearRegression
features = ["was_home", "opponent_team_strength"]
y_col = "total_points"
X = df[features].values
y = df[y_col]
reg = LinearRegression().fit(X, y)
reg.score(X, y)
reg.coef_
reg.intercept_
reg.predict(np.array([[True, 2]]))

array([3.88078752])

In [102]:
df["predicted_total_points"] = reg.predict(X)

In [105]:
manager_id = 6321588
url_manager = f'https://fantasy.premierleague.com/api/entry/{manager_id}/'
api_result_manager = requests.get(url_manager).json()

In [106]:
api_result_manager

{'id': 6321588,
 'joined_time': '2023-08-10T16:23:02.593523Z',
 'started_event': 1,
 'favourite_team': 7,
 'player_first_name': 'Mateusz',
 'player_last_name': 'Gimziński',
 'player_region_id': 172,
 'player_region_name': 'Poland',
 'player_region_iso_code_short': 'PL',
 'player_region_iso_code_long': 'POL',
 'years_active': 7,
 'summary_overall_points': 1612,
 'summary_overall_rank': 1535205,
 'summary_event_points': 19,
 'summary_event_rank': 1992081,
 'current_event': 29,
 'leagues': {'classic': [{'id': 7,
    'name': 'Chelsea',
    'short_name': 'team-7',
    'created': '2023-07-05T15:13:48.696125Z',
    'closed': False,
    'rank': None,
    'max_entries': None,
    'league_type': 's',
    'scoring': 'c',
    'admin_entry': None,
    'start_event': 1,
    'entry_can_leave': False,
    'entry_can_admin': False,
    'entry_can_invite': False,
    'has_cup': True,
    'cup_league': 2366059,
    'cup_qualified': True,
    'rank_count': 999735,
    'entry_percentile_rank': 15,
    'ent